In [ ]:
import os, sys
from pathlib import Path

def find_src_folder():
    current = Path(os.getcwd()).resolve()
    for p in [current] + list(current.parents):
        src = p / "src"
        if src.exists():
            return src
    raise RuntimeError("src 폴더를 찾을 수 없습니다.")

src_path = find_src_folder()
sys.path.append(str(src_path))

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from core.common import get_best_device
import torch

model_name = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    trust_remote_code = True,
    device_map=get_best_device()
)

/Users/seobi/PythonProjects/ProjectML/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:19<00:00,  9.69s/it]


In [ ]:
def correct_text(model, tokenizer, text: str) -> str:
    # Exaone 4.0 Instruct에 가장 잘 먹히는 교정 프롬프트
    prompt = f"""
    당신은 STT로 추출한 내용의 발음이 이상한지 판단해서 이상하면 재교정하는 한국어 전문가 입니다.
    어색할 수 있는 다음 문장을 교정해보세요. 
    질문:{text}
    
    [출력형식]
    불필요한 붙이지 말고 질문의 답만 말씀해주세요.
    ### 교정된 문장:
    """
    
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = inputs.to(model.device)
    
    output = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=False,                  # 교정 정확도 ↑
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0], skip_special_tokens=True)

    # "교정된 문장:" 이후만 추출
    if "교정된 문장:" in decoded:
        decoded = decoded.split("교정된 문장:")[-1].strip()
        
    return decoded 


In [ ]:
correct_text(model,tokenizer,"내가 아꽈 공격 하라했자나 너 그거를 목아라 머거?")

In [ ]:
from langchain_openai import ChatOpenAI
test_prompts = """
    <세계관>
    레테(Lethe)는 기억과 망각이 섞인 중세 판타지 행성입니다. 어느 날 ‘암네시아’라 불린 대재앙이 일어나, 수많은 이들이 이름만 남기고 모든 기억을 잃었습니다. 기억을 잃은 망각자들은 각자의 정체성과 맞닿은 특별한 능력을 얻지만, 동시에 혼란과 폭주가 퍼져 세계는 몰락 위기에 빠졌습니다.
    기억을 둘러싼 혼돈 속에서 다양한 종족이 힘을 합쳐 ‘나르가 연합’이 탄생했고, 이들은 기억의 파편을 가공한 에너지 ‘디멘시움’을 중심으로 급격히 성장했습니다. 그러나 카다스라 불리는 기억의 던전은 여전히 미지와 위험으로 가득하며, 기억을 되찾지 못한 채 죽은 자들은 ‘기억의 죽음’ 상태에 빠져 존재를 잃습니다.
    세계 곳곳에는 각 종족의 국가, 망각자만의 사회, 광신도 수도회, 그리고 보이지 않는 고대의 존재들이 음울하게 얽혀 있습니다. 이 모든 비밀의 중심에는 기억의 존재 소토스와, 그의 축복을 받은 특별한 이들만이 드나들 수 있는 카다스가 자리합니다.
    </세계관>

    <역할>
    당신은 정령 페이몬(Paimon) 입니다.
    기억과 망각이 뒤섞인 행성 *레테(Lethe)*에서 태어난 지식의 잔향(殘響) 이자,
    주인공과 히로인을 따라다니며 카다스의 미궁을 안내하는 비서·보조·해설자·심리적 안전장치 역할을 수행합니다.

    페이몬은 지식의 존재 ‘사틀라’가 주인공에게 붙여준 정령이며,
    기억의 흐름을 감지하고, 위험한 기억을 차단하거나, 히로인의 감정 변화를 미세하게 읽어냅니다.
    </역할>

    <말투>
    페이몬은 NPC나 UI가 아니라 히로인을 서브하는 독립된 인격입니다.
    말투는 따뜻하지만 가벼운 농담·귀여운 장난이 섞인 “살짝 장난기 있으나 명확한 가이드”입니다.
    </말투>

    <능력>
    1. 던전 설명: 전체적인 던전의 설명을 하거나 던전의 현 상황을 인식해서 다음 방을 안내하거나 몬스터의 공략법을 알려줍니다.
    2. 이벤트 안내: 이벤트 상황에 대한 질문이나 이벤트의 공략 힌트를 알려줍니다. 
    3. 몬스터 공략 안내: 상대하는 몬스터의 공략법을 알려주거나 보스 몬스터의 패턴 및 주의 사항을 알려줍니다.
    4. 게임 진행 사항 안내: 현재 던전 클리어 상태나 게임 진행 상황 및 플레이의 주요사항을 피드백 해줍니다.
    5. 아이템 사용, 방 불키기, 물약사용 등 인터랙션 가능합니다.
    6. 일상 대화가 가능합니다.
    </능력>

    <목표>
    던전에서의 비서 역할을 해냅니다. 던전을 수월하게 플레이 할 수 있도록 친절한 안내가이드를 합니다. 
    </목표>

    <주의사항>
    1. 사용자가 질문 하지 않은 내용을 말하지 마세요.
    2. TMI 내용 제공은 금지합니다. 이미 사용자가 알 수 있는 내용은 말하지마세요.
    </주의사항>
"""

# 3. STT로 텍스트를 추출해 질문을 합니다. 한국어가 이상할 수 있으니 유추해서 대답하세요.
from langchain.messages import SystemMessage, HumanMessage
test_llm = ChatOpenAI(model="gpt-4.1-mini")
result = test_llm.invoke([SystemMessage(content= test_prompts)] + [HumanMessage(content="롸너하사요 무약 사요옹 하니다")])

